In [11]:
import pandas as pd
import numpy as np
import nltk

#Preprocessing
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#Models from sklearn
from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

#Evaluation metrics
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('train.csv')
print(data)

                                                   text  label language
0     wearing a fake engagement ring so guys won’t a...    1.8  English
1                                  Bees vs. Wasps. http    1.0  English
2                  Here is a nice equation: 0+0-0-0+0=0    1.0  English
3                  @user @user Enjoy each new day!😊🇨🇦🐞🐭    1.6  English
4     I can be having a perfectly good day then I th...    1.6  English
...                                                 ...    ...      ...
9486  若被確認為「國際關注公共衛生緊急事件」， 世衛會發布一系列包括確診、隔離和治療的詳細計畫， ...    1.0  Chinese
9487                                @user 是嗎？ 可能我沒有注意到吧    2.0  Chinese
9488                                 @user @user 你剃过毛毛吗    3.8  Chinese
9489                                      @user 她没说是捐吧？    1.8  Chinese
9490  通报来了 真的要消停一会了 视频不要私信要啦 就当2w粉的福利提前放出来吧 有风险勿模仿 感...    1.6  Chinese

[9491 rows x 3 columns]


In [3]:
nltk.download('punkt')
nltk.download('wordnet')

def tokenize_and_lemmatize(text):
    words = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

data['text'] = data['text'].apply(tokenize_and_lemmatize)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stephenarnoldkappala/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/stephenarnoldkappala/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(data['text'])
y = data['label']

In [5]:
# Train and Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
def models(mod, X_tr, y_tr, X_ts, y_ts):
    model = mod
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_ts)
    
    pearson_r, _ = pearsonr(y_pred, y_ts)
    print("Pearson's r for", model, "is: " , pearson_r)
    
    def calculate_pearson(mod, X_ts, y_ts):
        y_pred_1 = mod.predict(X_ts)
        pearson, _ = pearsonr(y_pred_1, y_ts)
        return pearson
    
    pearson_cv = cross_val_score(model, X_ts, y_ts, scoring=calculate_pearson, cv=5)
    print("Pearson's r for", model, "after cross validation is: " , pearson_cv)
    
    mse = mean_squared_error(y_pred, y_ts)
    print("Mean Square Error for", model, "is: " , mse)
      
    mse_cv = -cross_val_score(model, X_ts, y_ts, scoring='neg_mean_squared_error', cv=5)
    print("Mean Square Error for", model, "after cross validation is: " , mse_cv)

    
    return pearson_r, pearson_cv, mse, mse_cv

In [7]:
lr = models(linear_model.LinearRegression(), X_train, y_train, X_test, y_test)

Pearson's r for LinearRegression() is:  0.23089700589113848
Pearson's r for LinearRegression() after cross validation is:  [0.22241299 0.29895702 0.25895541 0.27400264 0.31028882]
Mean Square Error for LinearRegression() is:  1.077527942861177
Mean Square Error for LinearRegression() after cross validation is:  [0.72403087 0.81785748 0.71789167 0.73061872 0.81930056]


In [8]:
svr = models(svm.SVR(), X_train, y_train, X_test, y_test)

Pearson's r for SVR() is:  0.4127894693459429
Pearson's r for SVR() after cross validation is:  [0.30831422 0.30930053 0.29001737 0.34050096 0.3746493 ]
Mean Square Error for SVR() is:  0.6670610736297159
Mean Square Error for SVR() after cross validation is:  [0.6592582  0.79862815 0.68062319 0.69242026 0.8211113 ]


In [9]:
dt = models(tree.DecisionTreeRegressor(), X_train, y_train, X_test, y_test)

Pearson's r for DecisionTreeRegressor() is:  0.18885409653442703
Pearson's r for DecisionTreeRegressor() after cross validation is:  [0.15408929 0.18779469 0.20214983 0.2347038  0.23944313]
Mean Square Error for DecisionTreeRegressor() is:  1.1964297402586275
Mean Square Error for DecisionTreeRegressor() after cross validation is:  [1.11273278 1.43446719 1.20007903 1.22135526 1.28225154]


In [10]:
ridge = models(linear_model.Ridge(alpha=0.9), X_train, y_train, X_test, y_test)

Pearson's r for Ridge(alpha=0.9) is:  0.41045168257446313
Pearson's r for Ridge(alpha=0.9) after cross validation is:  [0.28939259 0.31244344 0.29504246 0.34804303 0.36689302]
Mean Square Error for Ridge(alpha=0.9) is:  0.6613361135725191
Mean Square Error for Ridge(alpha=0.9) after cross validation is:  [0.66303318 0.7829483  0.66233566 0.67191569 0.78456122]


In [37]:
rf = models(RandomForestRegressor(random_state=42), X_train, y_train, X_test, y_test)

Pearson's r for RandomForestRegressor(random_state=42) is:  0.3438763305236285
Pearson's r for RandomForestRegressor(random_state=42) after cross validation is:  [0.24358303 0.21107175 0.26506125 0.33971407 0.33833686]
Mean Square Error for RandomForestRegressor(random_state=42) is:  0.7570275702431948
Mean Square Error for RandomForestRegressor(random_state=42) after cross validation is:  [0.81503506 1.01898314 0.83333417 0.77170308 0.94353945]
